----
# Setup

## Configure Colab File System

Make sure you shortcut the lfqa_266 folder to MyDrive!
> `lfqa_266` -> `Organize` -> `Add shortcut`



![](https://drive.google.com/uc?export=view&id=1sxFMcaAAWTGfFYyTlmvFZmEgfNdeU0Ks)

<br></br>


----
## Stay Connected to Colab Runtime
If you want to let your colab keep running while you're gone, copy-paste this into the editor console:
> - `option` + `command` + `i`
- `copy-paste` the snippet into the *console*, then hit `enter`
- check for printouts at 60s mark, then run your job

```javascript
function ClickConnect(){
    console.log("Click #1");
    document.querySelector("colab-toolbar-button#connect").click();
    console.log("Click #2");
    sleep(1000);
    document.querySelector("colab-toolbar-button#connect").click();
}
setInterval(ClickConnect,60000);
```

<br></br>


----
## Install Packages & Dependencies

In [1]:
# @title
import importlib

def install_if_needed(package_names):
    """
    Install one or more Python packages using pip if they are not already installed.

    Args:
        package_names (str or list): The name(s) of the package(s) to install.

    Returns:
        None
    """
    print('installing packages')
    if isinstance(package_names, str):
        package_names = [package_names]

    newly_installed = []
    for package_name in package_names:
        try:
            importlib.import_module(package_name)
            print(f"- {package_name} is already installed.")
        except ImportError:
            !pip install --quiet {package_name} --upgrade
            newly_installed.append(package_name)

    print(f"installed {', '.join(newly_installed)}")
    print(f"imported {', '.join([x for x in package_names if x not in newly_installed])}")


def running_in_colab():
    """
    Check if the Jupyter Notebook is running in Google Colab.

    Returns:
        bool: True if running in Google Colab, False otherwise.
    """
    try:
        import google.colab

        return True
    except ImportError:
        return False


def mount_to_drive():
    """
    Mount notebook to colab file directory
    """
    if running_in_colab():
      print('\tmounting to drive')
      from google.colab import drive
      drive.mount("/content/drive", force_remount=True)


def unpack_elasticsearch():
  """
  Get elasticsearch jar file if necessary
  """
  ![[ -d /content/elasticsearch-8.11.1 ]] && echo "- elasticsearch is already installed" \
                                           || wget "https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-8.11.1-linux-x86_64.tar.gz" \
                                                && tar -xzf "elasticsearch-8.11.1-linux-x86_64.tar.gz" 1> /dev/null && echo "- elasticsearch has been installed"

In [4]:
ROOT='/content/drive/MyDrive/W210 Capstone - Lyric Generation with Melody/'
packages = [
  "dill",
  "faiss",
  "accelerate",
  "apache-beam",
  "datasets",
  "elasticsearch",
  "faiss_gpu",
  "keybert",
  "evaluate",
  "nlp",
  "nltk",
  "python-dotenv",
  "rouge-score",
  "rouge_score",
  "sentencepiece",
  "transformers",
  "unicodedata",
  "yake",
]

# rip
install_if_needed(packages)
mount_to_drive()
unpack_elasticsearch()

# system stuff
import ast
from collections import Counter
from dotenv import dotenv_values
dotenv = dotenv_values(f"{ROOT}/.env")
import numpy as np
from keybert import KeyBERT
import pandas as pd
import random
from math import ceil,floor
import os
# memory fragmentation & error tracing
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
import re
import string
import sys
sys.path.append(ROOT)
import unicodedata
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# big boiz
from datasets import load_dataset, Dataset, DatasetDict, load_metric, load_dataset_builder
from huggingface_hub import login, logout
import nlp
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import cmudict
# Download the CMU Pronouncing Dictionary if not already downloaded
nltk.download('cmudict')
nltk.download('punkt')

# import tensorflow as tf
# from tensorflow import keras
import torch
from transformers import BartTokenizer, BartTokenizerFast, BartForConditionalGeneration
from transformers import T5Tokenizer, T5ForConditionalGeneration, TFT5ForConditionalGeneration
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import yake

installing packages
- dill is already installed.
- faiss is already installed.
- accelerate is already installed.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.16 requires dill>=0.3.8, but you have dill 0.3.1.1 which is incompatible.
- datasets is already installed.
- elasticsearch is already installed.
- keybert is already installed.
- evaluate is already installed.
- nlp is already installed.
- nltk is already installed.
- rouge_score is already installed.
- sentencepiece is already installed.
- transformers is already installed.
- unicodedata is already installed.
- yake is already installed.
installed apache-beam, faiss_gpu, python-dotenv, rouge-score
imported dill, faiss, accelerate, datasets, elasticsearch, keybert, evaluate, nlp, nltk, rouge_score, sentencepiece, transformers, unicodedata, yake
	mounting to drive
Mounted at /content/d

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
# !pip install music21
import music21
# from music21 import converter, corpus

In [10]:
xml = music21.converter.parse(f"{ROOT}loaf/generated_song_Silly-old-Billy.musicxml")

In [20]:
music21.note.Lyric()

<music21.note.Lyric number=1>

In [26]:
note = music21.note.Note()

In [33]:
lyric = music21.note.Lyric(text='hello')

In [38]:
help(lyric.style.textDecoration)

Help on NoneType object:

class NoneType(object)
 |  Methods defined here:
 |  
 |  __bool__(self, /)
 |      True if self else False
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.



In [39]:
te = music21.expressions.TextExpression('Bridge')
te.enclosure = music21.style.Enclosure.RECTANGLE

In [40]:
note.lyric = te

In [41]:
note.show()

SubConverterException: Cannot find a path to the 'mscore' file at /usr/bin/mscore3 -- download MuseScore

In [27]:
help(note)

Help on Note in module music21.note object:

class Note(NotRest)
 |  Note(pitch: 'str | int | Pitch | None' = None, *, name: 'str | None' = None, nameWithOctave: 'str | None' = None, **keywords)
 |  
 |  One of the most important music21 classes, a Note
 |  stores a single note (that is, not a rest or an unpitched element)
 |  that can be represented by one or more notational units -- so
 |  for instance a C quarter-note and a D# eighth-tied-to-32nd are both
 |  a single Note object.
 |  
 |  A Note knows both its total duration and how to express itself as a set of
 |  tied notes of different lengths. For instance, a note of 2.5 quarters in
 |  length could be half tied to eighth or dotted quarter tied to quarter.
 |  
 |  The first argument to the Note is the pitch name (with or without
 |  octave, see the introduction to :class:`music21.pitch.Pitch`).
 |  Further arguments can be specified as keywords (such as type, dots, etc.)
 |  and are passed to the underlying :class:`music21.du

In [24]:
xml.show('text')

{0.0} <music21.metadata.Metadata object at 0x7f3c79574490>
{0.0} <music21.stream.Part Effect Synthesizer, MR. ROBOTO                      ;Words and music by Dennis De Young>
    {0.0} <music21.instrument.Sampler 'P1: Effect Synthesizer, MR. ROBOTO                      ;Words and music by Dennis De Young: Effect Synthesizer'>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.layout.SystemLayout>
        {0.0} <music21.clef.BassClef>
        {0.0} <music21.tempo.MetronomeMark molto allegro Quarter=145>
        {0.0} <music21.key.KeySignature of 5 flats>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Rest whole>
    {4.0} <music21.stream.Measure 2 offset=4.0>
        {0.0} <music21.note.Note F>
        {0.5} <music21.note.Note F>
        {1.0} <music21.note.Note F>
        {1.5} <music21.note.Note F>
        {2.0} <music21.note.Note F>
        {3.0} <music21.note.Note F>
        {3.5} <music21.note.Note F>
    {8.0} <music21.stream.Measur

In [19]:
for n in xml.notes:
  print(n.lyric)

In [23]:
for n in xml.recurse().parts.notes:
  print(n)

# Functions

## Save Data

In [ ]:
'''
save_data():
'''
def save_data(dataset, hf_write_path, write_data=False, message="Uploading"):
    if not write_data:
        return False
    # write to hf
    try:
        dataset.push_to_hub(hf_write_path,
                            token=dotenv['hf_write_token_adam'],
                            commit_message=message)
    except Exception as e:
        print('save to hf failed')
        print(e)
        return False

    return True

## LYRLEN

### Syllable Counting Functions

In [ ]:
# instantiate dictionary for phonetic syllables
syllable_dict = cmudict.dict()
# constants to be used in manual syllable counting
VOWEL_RUNS = re.compile("[aeiouy]+", flags=re.I)
EXCEPTIONS = re.compile(
    # fixes trailing e issues:
    # smite, scared
    "[^aeiou]e[sd]?$|"
    # fixes adverbs:
    # nicely
    + "[^e]ely$",
    flags=re.I
)
ADDITIONAL = re.compile(
    # fixes incorrect subtractions from exceptions:
    # smile, scarred, raises, fated
    "[^aeioulr][lr]e[sd]?$|[csgz]es$|[td]ed$|"
    # fixes miscellaneous issues:
    # flying, piano, video, prism, fire, evaluate
    + ".y[aeiou]|ia(?!n$)|eo|ism$|[^aeiou]ire$|[^gq]ua",
    flags=re.I
)

In [ ]:
# syllable_dict

In [ ]:
'''
clean_lyrics():
        - hf mapping function to remove bad characters from lyrics
'''
def clean_lyrics(row):
    lyrics = row['lyrics']
    # remove lyric annotations, typically surrounded by
    lyrics = re.sub(r'\[{1,}.*?\]{1,}', '', lyrics)         # brackets
    lyrics = re.sub(r'\({1,}.*?\){1,}', '', lyrics)         # parentheses
    lyrics = re.sub(r'\*{1,}.*?\*{1,}', '', lyrics)         # asterisks
    # remove odd punctuation
    lyrics = lyrics.replace(u'\xa0', ' ')                   # &nbsp
    lyrics = lyrics.replace(u'\\xa0', ' ')                  # &nbsp
    lyrics = re.sub(r"[^a-zA-Z0-9 .,-[\n]]", r"", lyrics)  # characters except .,- \n
    lyrics = re.sub(r'\.{2,}', '.', lyrics)                 # .. ...
    # remove dashes & replace with space
    lyrics = lyrics.replace("_", "")
    # remove extra whitespace caused by above
    lyrics = re.sub(' +', ' ', lyrics)                      # repeating space chars
    lyrics = re.sub(r'\n+', '\n', lyrics).strip()           # repeating/extra newline chars
    lyrics = lyrics.replace('\n \n', '\n')                  # edge case
    # fix capitalization
    lyrics = lyrics.lower()
    # remove apostrophes that split during tokenization
    lyrics = lyrics.replace("'", "")

    # extra safe (trying to remove \xao char)
    lyrics = unicodedata.normalize("NFKD", lyrics)

    return {'clean_lyrics': lyrics}

'''
count_word_syllables_man():
    - manually count syllables using ruleset
'''
def count_word_syllables_man(word):
    vowel_runs = len(VOWEL_RUNS.findall(word))
    exceptions = len(EXCEPTIONS.findall(word))
    additional = len(ADDITIONAL.findall(word))
    return max(1, vowel_runs - exceptions + additional)

'''
count_word_syllables_cmu():
    - uses cmu pronunciation dictionary
'''
def count_word_syllables_cmu(word):
  return [len(list(y for y in x if y[-1].isdigit())) for x in syllable_dict[word.lower()]][0]

'''
count_word_syllables():
'''
def count_word_syllables(word):
    try:
        return count_word_syllables_cmu(word)
    except KeyError:
        return count_word_syllables_man(word)

'''
count_line_syllables():
'''
def count_line_syllables(line):
    tokens = word_tokenize(line)
    syllable_counts = [count_word_syllables(token) for token in tokens if token.isalpha()]
    return sum(syllable_counts)

'''
uppercase():
        - regex matching function
'''
def uppercase(matchobj):
    return matchobj.group(0).upper()


'''
capitalize():
'''
def capitalize(s):
    return re.sub('^([a-z])|[\.|\?|\!]\s*([a-z])|\s+([a-z])(?=\.)', uppercase, s)


'''
fix_capitalization():
'''
def fix_capitalization(s):
    if s.isupper():
        s = s.lower()
    return capitalize(s)


'''
remove_punctuation():
'''
def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))


'''
get_lyric_lines():
        - split on newline and ignore empty lines
'''
def get_lyric_lines(lyrics):
    return [line.strip() for line in lyrics.split('\n') if line != '']


'''
split_into_chunks():
        - generator helper function to split a list into chunks
'''
def split_into_chunks(list, chunk_size):
    for i in range(0, len(list), chunk_size):
        yield list[i:i + chunk_size]


### Lyrlen Processing

In [ ]:
'''
parse_lyrlen():
    - hf mapping function
'''
def rename_lyrlen_cols(row):
  return {'id': row['Unnamed: 0'], 'genre': row['tag']}

'''
create_lyrlen_inputs():
    - hf mapping function
'''
def create_lyrlen_inputs(row):
  sylls = count_line_syllables(row['clean_lyrics'])
  return  {'sylls': sylls, 'orig': f'len_{sylls}', 'target': row['clean_lyrics']}


'''
get_lyric_chunks():
            - hf mapping function to convert lyrics to chunks
'''
def get_lyric_chunks(row, chunk_size):
    return {'lyric_chunks': [' '.join(chunk) for chunk in list(split_into_chunks(get_lyric_lines(row['clean_lyrics']), chunk_size))]}


'''
explode_chunks():
            - batched hf mapping function to create rows for each lyric chunk in a song
'''
def explode_chunks(batch, group):
  obj = {}
  # add constant groupby for each chunk
  for g in group:
    obj[g] = [_g for i, _g in enumerate(batch[g]) for _ in batch['lyric_chunks'][i]]
  # add lyric chunk + id
  obj["clean_lyrics"] = [chunk for chunk_list in batch["lyric_chunks"] for chunk in chunk_list]
  obj["lyric_chunk_n"] = [i for chunk_list in batch["lyric_chunks"] for i, chunk in enumerate(chunk_list)]

  return obj


    # return {
    #     "id": [id for i, id in enumerate(batch["id"]) for _ in batch["lyric_chunks"][i]],
    #     "title": [title for i, title in enumerate(batch["title"]) for _ in batch["lyric_chunks"][i]],
    #     "genre": [genre for i, genre in enumerate(batch["genre"]) for _ in batch["lyric_chunks"][i]],
    #     "lyrics": [lyrics for i, lyrics in enumerate(batch["lyrics"]) for _ in batch["lyric_chunks"][i]],
    #     "clean_lyrics": [chunk for chunk_list in batch["lyric_chunks"] for chunk in chunk_list],
    #     "lyric_chunk_n": [i for chunk_list in batch["lyric_chunks"] for i, chunk in enumerate(chunk_list)],
    # }


'''
preprocess_lyrlen_data():
    - bread & butter
'''
def preprocess_lyrlen_data(dotenv, n_examples_slice=1e6, hf_write_path='lyrlen', write_data=False):
  # load data
  lyrics = load_dataset('amishshah/song_lyrics', split='train').flatten()
  # shuffle and grab randomized sample (param)
  lyrics_shuffled = lyrics.shuffle(seed=42)
  lyrics_slice = lyrics_shuffled.select(range(int(n_examples_slice)))
  # renamed to id & genre
  lyrics_slice = lyrics_slice.map(rename_lyrlen_cols, remove_columns=['Unnamed: 0', 'tag'])
  # clean lyrics & remove extra columns
  lyrics_slice = lyrics_slice.map(clean_lyrics, remove_columns=[col for col in lyrics_slice.features if col not in ['id', 'title', 'genre', 'lyrics', 'clean_lyrics']])
  # split lyrics into chunks
  lyrics_slice = lyrics_slice.map(get_lyric_chunks, fn_kwargs={'chunk_size': 1})
  # explode chunks into their own rows
  lyrics_slice = lyrics_slice.map(explode_chunks,
                                  batched=True,
                                  fn_kwargs={'group': ['id', 'title', 'genre', 'lyrics']},
                                  remove_columns=['lyric_chunks'])
  # create inputs
  lyrics_slice = lyrics_slice.map(create_lyrlen_inputs, remove_columns=['lyrics', 'clean_lyrics'])
  # remove large syllable lines
  lyrics_slice = lyrics_slice.filter(lambda example: example["sylls"] < 30)
  # create train/test/valid splits
  lyrics_slice_train_test = lyrics_slice.train_test_split(test_size=0.2)
  lyrics_slice_test_valid = lyrics_slice_train_test['test'].train_test_split(test_size=0.5)
  del lyrics_slice # save ram
  # write to csv
  lyrics_slice_train_test['train'].to_csv(dotenv['lyrlen_train'])
  lyrics_slice_test_valid['train'].to_csv(dotenv['lyrlen_test'])
  lyrics_slice_test_valid['test'].to_csv(dotenv['lyrlen_valid'])
  # save ram
  del lyrics_slice_train_test, lyrics_slice_test_valid
  # construct hf dataset w/ splits
  dataset = load_dataset('csv', data_files={'train': dotenv['lyrlen_train'], 'test': dotenv['lyrlen_test'], 'valid': dotenv['lyrlen_valid']}, delimiter=',')

  print('shmoney!') if save_data(dataset, hf_write_path, write_data) else print("booooooooo!")


  return dataset


## KW Syll Gen Data

In [ ]:
'''
create_kwsylgen_inputs():
    - hf mapping function
'''
def create_kwsylgen_inputs(row):
  sylls = count_line_syllables(row['clean_lyrics'])
  kw_extractor = yake.KeywordExtractor(n = 1, top = max(floor(sylls / 2), 3))
  kws = kw_extractor.extract_keywords(row['clean_lyrics'])
  kws = [kw[0] for kw in kws if len(kws) >= 1]
  prev_line_phrase = f"naturally follow previous lyric line '{row['prev_clean_lyrics']}' and " if row['prev_clean_lyrics'] != '' else ''
  orig = f'''input {' '.join(kws)}. len_{sylls} output'''

  return  {'sylls': sylls, 'keywords': kws, 'orig': orig, 'target': row['clean_lyrics']}


'''
preprocess_lyrlen_data():
    - bread & butter
'''
def preprocess_kwsylgen_data(dotenv, frac=0.1, hf_write_path='kwsylgen', write_data=False):
  df = pd.read_csv(dotenv['source_data'])
  # add in new lyric summaries / cleaned lyrics & merge
  summ_df = load_dataset('adamjweintraut/bart_cnn_lyric_summaries', split='etl').to_pandas()[['midi_id', 'clean_lyrics', 'lyric_summary_bartv2']]
  df = pd.merge(df, summ_df, on='midi_id')
  # load splits
  train_ids = pd.read_csv(f"{dotenv['processed']}/train_dataset.csv")[['midi_id', 'topic']]
  test_ids = pd.read_csv(f"{dotenv['processed']}/test_dataset.csv")[['midi_id', 'topic']]
  valid_ids = pd.read_csv(f"{dotenv['processed']}/valid_dataset.csv")[['midi_id', 'topic']]
  # create splits to save
  train_df, test_df, valid_df = tuple([pd.merge(df, id_set, on='midi_id') for id_set in [train_ids, test_ids, valid_ids]])
  # reduce
  train_df = train_df.sample(frac=frac, random_state=42)
  test_df = test_df.sample(frac=frac, random_state=42)
  valid_df = valid_df.sample(frac=frac, random_state=42)

  # save to csv first
  train_df.to_csv(dotenv[f'{hf_write_path}_train'])
  test_df.to_csv(dotenv[f'{hf_write_path}_test'])
  valid_df.to_csv(dotenv[f'{hf_write_path}_valid'])
  # delete to save ram
  del df, train_ids, test_ids, valid_ids, train_df, test_df, valid_df
  # load hf dataset
  dataset = load_dataset('csv', data_files={'train': dotenv[f'{hf_write_path}_train'],
                                           'test': dotenv[f'{hf_write_path}_test'],
                                           'valid': dotenv[f'{hf_write_path}_valid']},
                         delimiter=',')
  # define cols we don't need
  cols_to_axe=[
    'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 0_x',
    'msd_id', 'match_score', 'midi_filename', 'lpd_filename', 'lyrics_filename',
    'midi_end_time', 'midi_instruments', 'midi_program_numbers', 'lpd_resolution', 'lpd_tracks',
    'lpd_track_program_numbers', 'lpd_track_lengths', 'lyrics_time_events', 'lyrics_max_track_length',
    'artist', 'artist_terms', 'album', 'year_released', 'bpm', 'lyric_lang', 'syl_by_measure',
    'syl_count_by_measure', 'note_count_by_measure', 'measure_times', 'vocal_instrument_program', 'vocal_instrument_note_syl_corr',
    'vocal_instrument_name', 'note_onsets_by_measure', 'note_durs_by_measure', 'note_pitches_by_measure', 'note_velocities_by_measure', 'note_stresses_by_measure', 'lyric_summary_bart',
  ]

  ## clean & re-save
  for split in ['train', 'test', 'valid']:
    dataset[split] = dataset[split].map(clean_lyrics, remove_columns=cols_to_axe)
    dataset[split] = dataset[split].map(get_lyric_chunks, fn_kwargs={'chunk_size': 1}, remove_columns=['clean_lyrics'])
    dataset[split] = dataset[split].map(explode_chunks,
                                        batched=True,
                                        fn_kwargs={'group': ['midi_id', 'song_title', 'lyrics', 'genre', 'topic', 'lyric_summary_bartv2']},
                                        remove_columns=['lyric_chunks'])

    split_df = dataset[split].to_pandas()
    split_df['prev_clean_lyrics'] = split_df.sort_values(['midi_id', 'lyric_chunk_n']) \
                                            .groupby(['midi_id'])['clean_lyrics'] \
                                            .shift(1)
    split_df.fillna(value='', inplace=True)
    # reload so we can create inputs
    dataset[split] = Dataset.from_pandas(split_df)
    # create inputs for modeling
    print('creating inputs')
    dataset[split] = dataset[split].map(create_kwsylgen_inputs)

    dataset[split] = dataset[split].filter(lambda example: example["sylls"] < 30)
    dataset[split] = dataset[split].filter(lambda example: example["sylls"] > 2)
    dataset[split] = dataset[split].filter(lambda example: len(example["keywords"]) > 0)
    # write to csv
    dataset[split].to_csv(dotenv[f'{hf_write_path}_{split}'])

  # # write to hf
  login(dotenv['hf_write_token_adam'])
  dataset.push_to_hub(hf_write_path)

  return dataset

## LOAF

In [ ]:
def load_array_cols(row):
  for arr_col in ['syl_count_by_measure', 'note_onsets_by_measure', 'note_durs_by_measure', 'note_pitches_by_measure', 'note_stresses_by_measure', 'note_velocities_by_measure']:
    row[arr_col] = ast.literal_eval(row[arr_col])

  return row


'''
parse_loaf():
    - hf mapping function
    - utilizes existing tokens: https://github.com/huggingface/transformers/blob/main/src/transformers/models/bart/tokenization_bart.py
'''
def parse_loaf(row):
  # lines
  lines = row['clean_lyrics'].split('\n')
  # create keyword extractor obj
  kw_model = KeyBERT()
  # kw_extractor = yake.KeywordExtractor(n = 1, top = 3, dedupLim=0.8, dedupFunc='seqm')
  # extract keywords for training
  kws_per_line = [[kw for kw in kw_model.extract_keywords(line)] for line in lines]
  # sort?
  for kws in kws_per_line: kws.sort(key=lambda l: l[1])
  kws_per_line = [[kw[0] for kw in kws] for kws in kws_per_line if len(kws) >= 1]
  kws_string_per_line = [f"{', '.join(kws)}" for i, kws in enumerate(kws_per_line)]
  masks_per_line = [f"{', '.join(['<mask>']*len(kws))}" for i, kws in enumerate(kws_per_line)]
  # create plan prompt
  plan_prompt = f"Generate a plan for the lyrics given the song summary: {str(row['lyric_summary_bartv2'])}, genre: {str(row['genre'])}, and topic: {str(row['topic'])}. "
  # create dict to return w/ special token representations
  row_dict = {
      'midi_id': row['midi_id'],
      'orig_syl_tokens': " </s> ".join([f'len_{x}' for x in row["syl_count_by_measure"]]),
      'orig_note_onsets_tokens': " </s> ".join([' '.join([f'onset_{np.round(float(val), 1)} </s>' for val in line]) for line in row["note_onsets_by_measure"]]),
      'orig_note_dur_tokens': " </s> ".join([' '.join([f'dur_{np.round(float(val), 1)}' for val in line]) for line in row["note_durs_by_measure"]]),
      'orig_note_pitch_tokens': " </s> ".join([' '.join([f'pitch_{val}' for val in line]) for line in row["note_pitches_by_measure"]]),
      'orig_note_vel_tokens': " </s> ".join([' '.join([f'vel_{val}' for val in line]) for line in row["note_velocities_by_measure"]]),
      'orig_note_stress_tokens': " </s> ".join([' '.join([f'stress_{val}' for val in line]) for line in row["note_stresses_by_measure"]]),
      'plan_orig': f"{plan_prompt} {'. '.join(masks_per_line)}",
      'lines': lines,
      'kws_per_line': kws_per_line,
      'plan_target': f"{'. '.join(kws_string_per_line)}",
      'lyrics_target': " <P> ".join(row['clean_lyrics'].split('\n')),
  }

  return row_dict


'''
preprocess_loaf_data():
    - bread & butter
'''
def preprocess_loaf_data(dotenv, hf_base='adamjweintraut/loaf'):
  df = pd.read_csv(dotenv['source_data'])
  # add in new lyric summaries / cleaned lyrics & merge
  summ_df = load_dataset('adamjweintraut/bart_cnn_lyric_summaries', split='etl').to_pandas()[['midi_id', 'clean_lyrics', 'lyric_summary_bartv2']]
  df = pd.merge(df, summ_df, on='midi_id')
  # load splits
  train_ids = pd.read_csv(f"{dotenv['processed']}/train_dataset.csv")[['midi_id', 'topic']]
  test_ids = pd.read_csv(f"{dotenv['processed']}/test_dataset.csv")[['midi_id', 'topic']]
  valid_ids = pd.read_csv(f"{dotenv['processed']}/valid_dataset.csv")[['midi_id', 'topic']]
  # create splits to save
  train_df, test_df, valid_df = tuple([pd.merge(df, id_set, on='midi_id') for id_set in [train_ids, test_ids, valid_ids]])
  # save to csv first
  train_df.to_csv(dotenv['loaf_train'])
  test_df.to_csv(dotenv['loaf_test'])
  valid_df.to_csv(dotenv['loaf_valid'])
  # delete to save ram
  del df, train_ids, test_ids, valid_ids, train_df, test_df, valid_df
  # load hf dataset
  dataset = load_dataset('csv', data_files={'train': dotenv['loaf_train'], 'test': dotenv['loaf_test'], 'valid': dotenv['loaf_valid']}, delimiter=',')
  # define cols we don't need
  cols_to_axe=[
    'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Unnamed: 0_x',
    'msd_id', 'match_score', 'midi_filename', 'lpd_filename', 'lyrics_filename',
    'midi_end_time', 'midi_instruments', 'midi_program_numbers', 'lpd_resolution', 'lpd_tracks',
    'lpd_track_program_numbers', 'lpd_track_lengths', 'lyrics_time_events', 'lyrics_max_track_length',
    'artist', 'artist_terms', 'album', 'year_released', 'bpm', 'lyric_lang', 'syl_by_measure',
    'syl_count_by_measure', 'note_count_by_measure', 'measure_times', 'vocal_instrument_program', 'vocal_instrument_note_syl_corr',
    'vocal_instrument_name', 'note_onsets_by_measure', 'note_durs_by_measure', 'note_pitches_by_measure', 'note_velocities_by_measure', 'note_stresses_by_measure', 'clean_lyrics',
  ]
  ## clean & re-save
  # train
  dataset['train'] = dataset['train'].map(clean_lyrics)
  dataset['train'] = dataset['train'].map(load_array_cols)
  dataset['train'] = dataset['train'].map(parse_loaf, remove_columns=cols_to_axe)
  dataset['train'].to_csv(dotenv['loaf_train'])
  # test
  dataset['test'] = dataset['test'].map(clean_lyrics)
  dataset['test'] = dataset['test'].map(load_array_cols)
  dataset['test'] = dataset['test'].map(parse_loaf, remove_columns=cols_to_axe)
  dataset['test'].to_csv(dotenv['loaf_test'])
  # valid
  dataset['valid'] = dataset['valid'].map(clean_lyrics)
  dataset['valid'] = dataset['valid'].map(load_array_cols)
  dataset['valid'] = dataset['valid'].map(parse_loaf, remove_columns=cols_to_axe)
  dataset['valid'].to_csv(dotenv['loaf_valid'])

  # write to hf
  login(dotenv['hf_write_token_adam'])
  dataset.push_to_hub(hf_base)

  return dataset


----
# Run

In [ ]:
# make sure we have updated environment variables (root defined in setup)
dotenv = dotenv_values(f'{ROOT}/.env')

# login to HF
login(dotenv['hf_write_token_adam'])
login(dotenv['hf_read_token_adam'])

## create our datasets (on hf & csv)
kwsg = preprocess_kwsylgen_data(dotenv, frac=1.0)
# loaf_data = preprocess_loaf_data(dotenv)
# lyrlen_data = preprocess_lyrlen_data(dotenv, n_examples_slice=3e4)
# lyrlen_line_data = preprocess_lyrlen_data(dotenv, n_examples_slice=5e3, hf_write_path='lyrlen', write_data=True)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating etl split:   0%|          | 0/5682 [00:00<?, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/986 [00:00<?, ?B/s]

In [ ]:

  # login(dotenv['hf_write_token_adam'])
  # lyrlen_data.push_to_hub('adamjweintraut/lyrlen_data')

<br>

<br>

<br>

<br>

## Testing

In [ ]:
one_row = dataset['train'][0]

NameError: name 'dataset' is not defined

In [ ]:
one_row.keys()

NameError: name 'one_row' is not defined

In [ ]:
one_row['syl_count_by_measure'] = ast.literal_eval(one_row['syl_count_by_measure'])
# one_row['note_onsets_by_measure'] = ast.literal_eval(one_row['note_onsets_by_measure'])
one_row['note_durs_by_measure'] = ast.literal_eval(one_row['note_durs_by_measure'])
one_row['note_pitches_by_measure'] = ast.literal_eval(one_row['note_pitches_by_measure'])
one_row['note_stresses_by_measure'] = ast.literal_eval(one_row['note_stresses_by_measure'])
one_row['note_velocities_by_measure'] = ast.literal_eval(one_row['note_velocities_by_measure'])

In [ ]:
one_row["note_onsets_by_measure"][:2]

[[11.02294921875, 11.26953125, 11.4990234375, 11.9482421875, 12.3974609375],
 [12.6416015625,
  14.0185546875,
  14.2578125,
  14.47265625,
  14.70703125,
  15.4052734375,
  15.654296875,
  16.083984375,
  16.2841796875]]

In [ ]:
"<P> " + " <P> ".join([' '.join([f'onset_{np.round(float(val),1)}' for val in line]) for line in one_row["note_onsets_by_measure"]])

'<P> onset_11.0 onset_11.3 onset_11.5 onset_11.9 onset_12.4 <P> onset_12.6 onset_14.0 onset_14.3 onset_14.5 onset_14.7 onset_15.4 onset_15.7 onset_16.1 onset_16.3 <P> onset_16.7 onset_17.3 onset_17.8 onset_18.0 onset_18.3 onset_18.5 onset_18.7 onset_19.2 <P> onset_19.5 onset_20.4 onset_20.6 onset_21.1 onset_21.3 onset_21.8 <P> onset_22.0 onset_25.3 onset_25.6 onset_25.8 onset_26.0 <P> onset_26.4 onset_29.0 onset_29.3 onset_29.5 onset_29.7 onset_30.4 onset_30.7 onset_31.1 onset_31.4 <P> onset_31.7 onset_32.1 onset_32.4 onset_32.6 onset_33.0 onset_33.3 onset_33.5 <P> onset_33.9 onset_35.2 onset_35.4 onset_35.6 onset_36.1 onset_36.3 onset_36.7 <P> onset_37.0 onset_40.8 onset_41.0 onset_41.3 onset_41.7 onset_42.0 onset_42.8 onset_43.1 onset_43.4 onset_43.7 <P> onset_44.1 onset_44.5 onset_44.8 onset_45.0 onset_45.5 onset_45.7 onset_46.2 <P> onset_46.4 onset_48.3 onset_48.5 onset_48.7 onset_49.0 onset_49.2 onset_49.5 onset_49.9 onset_50.2 <P> onset_50.9 onset_51.8 onset_52.0 onset_52.3 onset

In [ ]:
one_row['lyrics']

'Goodbye Norma Jean\nThough I never knew you at all\nYou had the grace to hold yourself\nWhile those around you crawled\nThey crawled out of the woodwork\nAnd they whispered into your brain\nThey set you on the treadmill\nAnd they made you change your name\nAnd it seems to me you lived your life\nLike a candle in the wind\nNever knowing who to cling to\nWhen the rain set in\nAnd I would have\nliked to have known you\nBut I was just a kid\nYour candle burned out long before\nYour legend ever did\nLoneliness was tough\nThe toughest role you ever played\nHollywood created a superstar\nAnd pain was the price you paid\nEven when you died\nOh the press still hounded you\nAll the papers had to say\nWas that Marilyn was found in the nude\nAnd it seems to me you lived your life\nLike a candle in the wind\nNever knowing who to cling to\nWhen the rain set in\nAnd I would have\nliked to have known you\nBut I was just a kid\nYour candle burned out long before\nYour legend ever did\nGoodbye Norma Je

In [ ]:
"<P> " + " <P> ".join(one_row['lyrics'].split('\n'))

'<P> Goodbye Norma Jean <P> Though I never knew you at all <P> You had the grace to hold yourself <P> While those around you crawled <P> They crawled out of the woodwork <P> And they whispered into your brain <P> They set you on the treadmill <P> And they made you change your name <P> And it seems to me you lived your life <P> Like a candle in the wind <P> Never knowing who to cling to <P> When the rain set in <P> And I would have <P> liked to have known you <P> But I was just a kid <P> Your candle burned out long before <P> Your legend ever did <P> Loneliness was tough <P> The toughest role you ever played <P> Hollywood created a superstar <P> And pain was the price you paid <P> Even when you died <P> Oh the press still hounded you <P> All the papers had to say <P> Was that Marilyn was found in the nude <P> And it seems to me you lived your life <P> Like a candle in the wind <P> Never knowing who to cling to <P> When the rain set in <P> And I would have <P> liked to have known you <P>

In [ ]:
 "<P> " + " <P> ".join([' '.join([f'stress_{val}' for val in line]) for line in one_row["note_stresses_by_measure"]]),

('<P> stress_1 stress_1 stress_1 stress_1 stress_0 <P> stress_0 stress_2 stress_2 stress_2 stress_2 stress_2 stress_2 stress_2 stress_1 <P> stress_1 stress_1 stress_2 stress_1 stress_1 stress_1 stress_2 stress_1 <P> stress_0 stress_1 stress_1 stress_1 stress_1 stress_1 <P> stress_2 stress_2 stress_2 stress_2 stress_2 <P> stress_2 stress_2 stress_2 stress_2 stress_2 stress_2 stress_2 stress_2 stress_1 <P> stress_2 stress_1 stress_2 stress_2 stress_2 stress_2 stress_2 <P> stress_1 stress_1 stress_2 stress_2 stress_2 stress_2 stress_2 <P> stress_1 stress_2 stress_2 stress_2 stress_1 stress_2 stress_1 stress_2 stress_2 stress_2 <P> stress_2 stress_2 stress_1 stress_2 stress_1 stress_1 stress_1 <P> stress_1 stress_2 stress_2 stress_2 stress_1 stress_1 stress_2 stress_2 stress_2 <P> stress_2 stress_2 stress_2 stress_2 stress_2 <P> stress_2 stress_2 stress_2 stress_2 <P> stress_0 stress_2 stress_2 stress_2 <P> stress_2 stress_2 stress_2 stress_1 stress_2 stress_2 <P> stress_2 stress_1 stress_

In [ ]:
 "<P> " + " <P> ".join([' '.join([f'pitch_{val}' for val in line]) for line in one_row["note_pitches_by_measure"]]),


('<P> pitch_56 pitch_56 pitch_56 pitch_56 pitch_54 <P> pitch_56 pitch_52 pitch_54 pitch_56 pitch_52 pitch_61 pitch_61 pitch_62 pitch_61 <P> pitch_59 pitch_59 pitch_61 pitch_59 pitch_61 pitch_59 pitch_59 pitch_52 <P> pitch_52 pitch_56 pitch_57 pitch_56 pitch_54 pitch_56 <P> pitch_49 pitch_52 pitch_54 pitch_56 pitch_59 <P> pitch_59 pitch_52 pitch_54 pitch_56 pitch_52 pitch_61 pitch_61 pitch_62 pitch_61 <P> pitch_59 pitch_59 pitch_61 pitch_61 pitch_61 pitch_59 pitch_59 <P> pitch_52 pitch_56 pitch_56 pitch_57 pitch_56 pitch_54 pitch_56 <P> pitch_49 pitch_56 pitch_57 pitch_59 pitch_59 pitch_59 pitch_56 pitch_59 pitch_59 pitch_59 <P> pitch_56 pitch_56 pitch_57 pitch_59 pitch_59 pitch_59 pitch_61 <P> pitch_52 pitch_56 pitch_57 pitch_59 pitch_59 pitch_59 pitch_59 pitch_59 pitch_61 <P> pitch_59 pitch_57 pitch_56 pitch_56 pitch_54 <P> pitch_54 pitch_59 pitch_59 pitch_61 <P> pitch_61 pitch_61 pitch_63 pitch_64 <P> pitch_63 pitch_63 pitch_61 pitch_59 pitch_61 pitch_61 <P> pitch_56 pitch_56 pitch_5

In [ ]:
"<P> " + " <P> ".join([' '.join([f'vel_{val}' for val in line]) for line in one_row["note_velocities_by_measure"]]),


('<P> vel_103 vel_103 vel_107 vel_101 vel_90 <P> vel_92 vel_126 vel_127 vel_111 vel_114 vel_112 vel_112 vel_113 vel_106 <P> vel_109 vel_109 vel_112 vel_98 vel_101 vel_100 vel_113 vel_97 <P> vel_89 vel_97 vel_99 vel_101 vel_105 vel_97 <P> vel_113 vel_114 vel_120 vel_120 vel_120 <P> vel_125 vel_127 vel_121 vel_127 vel_123 vel_127 vel_123 vel_115 vel_110 <P> vel_118 vel_104 vel_114 vel_123 vel_120 vel_113 vel_120 <P> vel_98 vel_107 vel_113 vel_114 vel_114 vel_113 vel_117 <P> vel_105 vel_127 vel_119 vel_127 vel_96 vel_127 vel_109 vel_125 vel_116 vel_119 <P> vel_127 vel_126 vel_100 vel_113 vel_96 vel_110 vel_110 <P> vel_104 vel_127 vel_115 vel_126 vel_108 vel_108 vel_123 vel_112 vel_127 <P> vel_127 vel_118 vel_123 vel_126 vel_118 <P> vel_117 vel_127 vel_114 vel_127 <P> vel_88 vel_127 vel_123 vel_127 <P> vel_123 vel_113 vel_116 vel_107 vel_121 vel_113 <P> vel_126 vel_109 vel_127 vel_109 vel_118 vel_120 vel_117 vel_96 <P> vel_110 vel_105 vel_113 vel_107 vel_109 vel_109 <P> vel_107 vel_127 vel